In [34]:
!pip install selenium
!apt-get update
!apt install -yqq xvfb chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (148 kB/s)
Reading package lists... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup

# Khởi tạo các tùy chọn cho trình duyệt
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--incognito")

In [36]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PBL7/CrawlGoodread

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PBL7/CrawlGoodread


In [37]:
import pandas as pd

df = pd.read_csv("books_url.csv")
df.head()

,book_link,title,author_link,author_name
0,https://www.goodreads.com/book/show/197577126-...,This Summer Will Be Different,https://www.goodreads.com/author/show/21483807...,Carley Fortune
1,https://www.goodreads.com/book/show/199797582-...,The Paradise Problem,https://www.goodreads.com/author/show/6556689....,Christina Lauren
2,https://www.goodreads.com/book/show/136276877-...,The Last Murder at the End of the World,https://www.goodreads.com/author/show/17160667...,Stuart Turton
3,https://www.goodreads.com/book/show/199798953-...,One Perfect Couple,https://www.goodreads.com/author/show/9013543....,Ruth Ware
4,https://www.goodreads.com/book/show/195790670-...,If Something Happens to Me,https://www.goodreads.com/author/show/19976903...,Alex Finlay


In [41]:
import time

# Tạo danh sách books
books = []
error_books = []

# Duyệt qua từng URL
for url in df['book_link']:
    attempts = 0
    while attempts < 5:  # Thử lại tối đa 5 lần
        try:
            # Khởi tạo trình duyệt Selenium
            browser = webdriver.Chrome(options=chrome_options)
            # Mở trang web
            browser.get(url)

            # Chờ cho trang web render hoàn tất
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.Button.Button--inline.Button--small')))

            # Tìm và click vào phần tử trên trang web
            browser.execute_script("document.querySelector('button.Button.Button--inline.Button--small[aria-label=\"Book details and editions\"]').click();")

            # Chờ cho trang web render lại sau khi click
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.EditionDetails')))

            # Lấy HTML sau khi click
            html = browser.page_source

            # Sử dụng BeautifulSoup để parse HTML
            soup = BeautifulSoup(html, 'html.parser')

            # Trích xuất URL của hình ảnh
            image_url = soup.find('div', class_='BookCover__image').find('img')['src']

            # Trích xuất tiêu đề
            title_without_series = soup.find('h1', class_='Text__title1', attrs={'data-testid': 'bookTitle'}).text.strip()

            # Trích xuất mô tả
            description = soup.find('div', class_='DetailsLayoutRightParagraph__widthConstrained').text.strip()

            # Trích xuất quốc gia
            country_element = soup.find('div', class_='WorkDetails')
            country = ""
            if country_element:
                country_setting = country_element.find('dt', text='Setting')
                country = country_setting.find_next_sibling('dd').find('div', class_='TruncatedContent__text').text.strip() if country_setting else ""

            # Tìm phần tử <div class="EditionDetails">
            edition_details = soup.find('div', class_='EditionDetails')

            # Trích xuất thông tin ISBN13
            isbn13_element = edition_details.find('dt', text='ISBN')
            isbn13 = isbn13_element.find_next_sibling('dd').text.strip().split()[0] if isbn13_element else ""

            # Trích xuất thông tin ISBN
            isbn_element = edition_details.find('span', class_='Text__subdued')
            isbn = isbn_element.text.strip().split()[1].strip('()') if isbn_element else ""

            # Trích xuất thông tin nhà xuất bản
            publisher_element = edition_details.find('dt', text='Published')
            if not publisher_element:
                publisher_element = edition_details.find('dt', text='Expected publication')
            publisher = publisher_element.find_next_sibling('dd').text.strip().split(' by ')[-1] if publisher_element else ""

            # Trích xuất thông tin ngày phát hành
            publish_date = publisher_element.find_next_sibling('dd').text.strip().split(' by ')[0] if publisher_element else ""

            # Trích xuất thông tin ngôn ngữ
            language_element = edition_details.find('dt', text='Language')
            language = language_element.find_next_sibling('dd').text.strip() if language_element else ""

            # Trích xuất thông tin số trang
            pages_element = edition_details.find('dt', text='Format')
            pages = pages_element.find_next_sibling('dd').text.strip().split(',')[0].split()[0] if pages_element else ""

            # Thêm thông tin của cuốn sách vào danh sách
            books.append({
                "isbn13": isbn13,
                "isbn": isbn,
                "title_without_series": title_without_series,
                "description": description,
                "image_url": image_url,
                "country": country,
                "publisher": publisher,
                "publish_date": publish_date,
                "language": language,
                "pages": pages
            })

            # In ra thông tin đã trích xuất
            print("URL:", url)
            print("ISBN13:", isbn13)
            print("ISBN:", isbn)
            print("Title Without Series:", title_without_series)
            print("Description:", description)
            print("Image URL:", image_url)
            print("Country:", country)
            print("Publisher:", publisher)
            print("Publish Date:", publish_date)
            print("Language:", language)
            print("Pages:", pages)
            print()

            break  # Nếu không có exception thì thoát khỏi vòng lặp

        except TimeoutException:
            print(f"TimeoutException occurred while trying to access {url}")
            attempts += 1
            time.sleep(5)  # Chờ 5 giây trước khi thử lại
        except NoSuchElementException:
            print(f"No such element exception occurred while trying to access {url}")
            attempts += 1
            time.sleep(5)  # Chờ 5 giây trước khi thử lại
        finally:
            # Đóng trình duyệt
            browser.quit()

    if attempts == 5:
        print(f"Reached maximum number of attempts (5) for URL: {url}. Skipping to next URL.")
        error_books.append(url)
if len(error_books) > 0:
    print(error_books)

<ipython-input-41-6fd994d386e4>:52: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  isbn13_element = edition_details.find('dt', text='ISBN')
<ipython-input-41-6fd994d386e4>:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  publisher_element = edition_details.find('dt', text='Published')
<ipython-input-41-6fd994d386e4>:62: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  publisher_element = edition_details.find('dt', text='Expected publication')
<ipython-input-41-6fd994d386e4>:69: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_element = edition_details.find('dt', text='Language')
<ipython-input-41-6fd994d386e4>:73: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pages_element = edition_details.find('dt', text='

URL: https://www.goodreads.com/book/show/197577126-this-summer-will-be-different
ISBN13: 9780593638880
ISBN: 0593638883
Title Without Series: This Summer Will Be Different
Description: This summer they’ll keep their promise. This summer they won’t give into temptation. This summer will be different.Lucy is the tourist vacationing at a beach house on Prince Edward Island. Felix is the local who shows her a very good time. The only problem: Lucy doesn’t know he’s her best friend’s younger brother. Lucy and Felix’s chemistry is unreal, but the list of reasons why they need to stay away from each other is long, and they vow to never repeat that electric night again.It’s easier said than done.Each year, Lucy escapes to PEI for a big breath of coastal air, fresh oysters and crisp vinho verde with her best friend, Bridget. Every visit begins with a long walk on the beach, beneath soaring red cliffs and a golden sun. And every visit, Lucy promises herself she won’t wind up in Felix’s bed. Agai

<ipython-input-41-6fd994d386e4>:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  country_setting = country_element.find('dt', text='Setting')


URL: https://www.goodreads.com/book/show/200338241-the-dixon-rule
ISBN13: 9781990101199
ISBN: 1990101194
Title Without Series: The Dixon Rule
Description: Diana Dixon has a lot going on this summer. She’s rehearsing for a ballroom dance competition, juggling two jobs, and dealing with an ex-boyfriend who can’t take the hint it’s over. Yet despite all that, she still has plenty of time and energy to tell Shane Lindley to screw off.Shane just moved into her apartment building and seems dedicated to sleeping his way through her entire cheerleading squad. Sure, he’s a tall, gorgeous hockey player, but he’s messing with her turf. This calls for some ground rules: no parties in her apartment, leave her teammates alone, and—most importantly—leave her alone.What Diana doesn’t realize is that Shane’s sick of hookups and tired of being on the rebound after his long-term girlfriend called it quits. He wants a relationship. And when his ex comes back into the picture, he pretends he has one to mak

In [42]:
df = df.join(pd.DataFrame(books))
df.head()

,book_link,title,author_link,author_name,isbn13,isbn,title_without_series,description,image_url,country,publisher,publish_date,language,pages
0,https://www.goodreads.com/book/show/197577126-...,This Summer Will Be Different,https://www.goodreads.com/author/show/21483807...,Carley Fortune,9780593638880,0593638883,This Summer Will Be Different,This summer they’ll keep their promise. This s...,https://images-na.ssl-images-amazon.com/images...,,Berkley,"May 7, 2024",English,320
1,https://www.goodreads.com/book/show/199797582-...,The Paradise Problem,https://www.goodreads.com/author/show/6556689....,Christina Lauren,9781668017722,1668017725,The Paradise Problem,"Christina Lauren, returns with a delicious new...",https://images-na.ssl-images-amazon.com/images...,,Gallery Books,"May 14, 2024",English,352
2,https://www.goodreads.com/book/show/136276877-...,The Last Murder at the End of the World,https://www.goodreads.com/author/show/17160667...,Stuart Turton,9781728254654,1728254655,The Last Murder at the End of the World,From the bestselling author of The 7 1/2 Death...,https://images-na.ssl-images-amazon.com/images...,,Sourcebooks Landmark,"May 21, 2024",English,400
3,https://www.goodreads.com/book/show/199798953-...,One Perfect Couple,https://www.goodreads.com/author/show/9013543....,Ruth Ware,9781668025598,1668025590,One Perfect Couple,Harkening to Agatha Christie’s classic And The...,https://images-na.ssl-images-amazon.com/images...,,Scout Press,"May 21, 2024",English,400
4,https://www.goodreads.com/book/show/195790670-...,If Something Happens to Me,https://www.goodreads.com/author/show/19976903...,Alex Finlay,9781250863799,1250863791,If Something Happens to Me,"For the past five years, Ryan Richardson has r...",https://images-na.ssl-images-amazon.com/images...,,Minotaur Books,"May 28, 2024",English,336


In [43]:
df.to_csv('books_detail.csv', index=False, encoding='utf-8-sig')